In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [4]:
model="meta-llama/Llama-2-7b-chat-hf"

In [6]:
tokenizer=AutoTokenizer.from_pretrained(model,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
model=AutoModelForCausalLM.from_pretrained(model,
                                           device_map='auto',
                                           torch_dtype=torch.float16,
                                           use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [8]:
pipeline=transformers.pipeline("text-generation",
                               model=model,
                               tokenizer=tokenizer,
                               torch_dtype=torch.bfloat16,
                               device_map="auto",
                               # Maximum number of tokens to generate, a word is generally 2-3 tokens (minimum: 1)
                               max_new_tokens=512,
                               #Minimum number of tokens to generate. To disable this set it to -1. A word is generally 2-3 tokens (minimum 1)
                               min_new_tokens=-1,
                               #Adjusts the randomness of output
                               temperature=0.75,
                               do_sample=True,
                               top_k=30,
                               num_return_sequences=1,
                               eos_token_id=tokenizer.eos_token_id)

https://replicate.com/meta/llama-2-7b-chat

In [9]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [10]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


" and the Philosopher's Stone by J.K. Rowling.  Please include the main characters, plot, and themes of the book.  Thank you.\n\nSure! Here is a summary of the book Harry Potter and the Philosopher's Stone by J.K. Rowling:\n\nMain Characters:\n\n* Harry Potter: The protagonist of the story, an orphaned boy who discovers he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry.\n* Hermione Granger: A fellow student at Hogwarts and one of Harry's closest friends.\n* Ron Weasley: Another student at Hogwarts and Harry's other close friend.\n\nPlot:\n\n* Harry discovers that he is a wizard and begins attending Hogwarts School of Witchcraft and Wizardry.\n* Harry and his friends uncover a plot by the evil wizard, Lord Voldemort, to steal the Philosopher's Stone, a powerful magical object that can grant eternal life.\n* Harry and his friends must navigate the challenges of school and friendship while also trying to stop Voldemort and his followers from obtaining the Phil

In [11]:
B_INST, E_INST= "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<<SYS>>\n\n"

In [12]:
DEFAULT_SYSTEM_PROMPT="""\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
"""

In [13]:
instruction = "Who won the T20 Cricket World Cup 2022"

In [14]:
SYSTEM_PROMPT=B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

In [15]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [16]:
template = B_INST + SYSTEM_PROMPT + instruction + E_INST

In [17]:
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nWho won the T20 Cricket World Cup 2022[/INST]"

# Text Translation

In [18]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [20]:
SYSTEM_PROMPT=B_SYS+DEFAULT_SYSTEM_PROMPT+E_SYS

In [21]:
SYSTEM_PROMPT

"<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\n"

In [22]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [23]:
template

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n\n<<SYS>>\n\nConvert the following text from English to French: \n\n{text}[/INST]"

In [24]:
print(template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.

<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [25]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [26]:
text ="How are you"

In [27]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [28]:
LLM_Chain.run(text)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'  Bonjour! I\'m here to help you with any questions you may have. However, I must inform you that I cannot provide a direct translation of "How are you" from English to French as it is a common English phrase that does not have an direct equivalent in French.\n\nIn English, "How are you?" is a common greeting used to ask about someone\'s well-being or health. In French, there are several ways to ask about someone\'s well-being, depending on the context and level of formality. Here are a few options:\n\n* "Comment vas-tu?" (formal) - This means "How are you?" in a formal setting or when speaking to someone you don\'t know well.\n* "Comment ça va?" (informal) - This is a more casual way of asking how someone is doing, and is commonly used among friends or acquaintances.\n* " comment vas-ce?" (informal) - This is another way to ask how someone is doing, and is similar to "Comment ça va?".\n\nI hope this helps! If you have any other questions, please feel free to ask.'

## Custom

In [29]:
instruction = "Convert the following text from English to French: \n\n{text}"

In [30]:
CUSTOM_SYSTEM_PROMPT="You are an advanced assistant that excels at translation."

In [31]:
SYSTEM_PROMPT=B_SYS+CUSTOM_SYSTEM_PROMPT+E_SYS

In [32]:
SYSTEM_PROMPT

'<<SYS>>\nYou are an advanced assistant that excels at translation.\n<<SYS>>\n\n'

In [33]:
template=B_INST+SYSTEM_PROMPT+instruction+E_INST

In [34]:
print(template)

[INST]<<SYS>>
You are an advanced assistant that excels at translation.
<<SYS>>

Convert the following text from English to French: 

{text}[/INST]


In [36]:
prompt=PromptTemplate(input_variables=["text"], template=template)

In [40]:
text ="How are you?"

In [38]:
LLM_Chain=LLMChain(llm=llm, prompt=prompt)

In [41]:
print(LLM_Chain.run(text))

  Sure! Here is the translation of "How are you?" from English to French:

Comment vas-tu?
